In [4]:
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
import sys
sys.path.insert(1, "/home/oru2/project/project")

In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import time
# import matplotlib.pyplot as plt
import attacks
from privacy_accountant import PrivacyAccountant
from tqdm import tqdm
from torchvision import datasets, transforms

from collections import Counter
import torch
from torch.utils.data import DataLoader, TensorDataset


In [7]:
import pandas as pd

data = pd.read_csv("../imdb_data/IMDB Dataset.csv")

In [26]:
#credit: https://www.kaggle.com/code/m0hammadjavad/imdb-sentiment-classifier-pytorch/notebook
import nltk
import os
import spacy
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')

nlp = spacy.load("en_core_web_sm")

stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    tokens = word_tokenize(text.lower())
    
    filtered_tokens = [word for word in tokens if word.isalpha() and word not in stop_words]

    doc = nlp(' '.join(filtered_tokens))
    
    lemmetized_tokens = [token.lemma_ for token in doc]
    
    return ' '.join(lemmetized_tokens)

if os.path.exists("../imdb_data/IMDB Dataset_with_cleaned_reviews.csv"):
    data = pd.read_csv("../imdb_data/IMDB Dataset_with_cleaned_reviews.csv")
else:  
    data["cleaned_reviews"] = data["review"].apply(preprocess_text)
    data.to_csv("../imdb_data/IMDB Dataset_with_cleaned_reviews.csv")

data["sentiment"] = data["sentiment"].map({"positive": 1, "negative": 0})

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data["cleaned_reviews"], data["sentiment"], test_size=0.2, random_state=42)
print(f'Training samples: {len(X_train)}, Test samples: {len(X_test)}')


# Create a vocabulary based on the training data
def build_vocab(texts):
    
    # this makes a dict with unique words and their count as the value
    # although this is not going to be used directly, it only gives us unique words without repeatition
    counter = Counter()
    for text in texts:
        counter.update(text.split())
        
    # this makes a dict of unique words and their index as the value
    vocab = {word: idx for idx, (word, _) in enumerate(counter.items(), 1)}  # Reserve index 0 for padding
    
    # this is a convention which is going to be used to convert batches to a fixed size
    vocab['<PAD>'] = 0
    
    return vocab

# Build the vocabulary
vocab = build_vocab(X_train)

# Encode text sequences into integer sequences
def encode_text(text, vocab, max_length=200):
    tokens = text.split()
    encoded = [vocab.get(token, 0) for token in tokens]  # 0 for unknown tokens
    if len(encoded) < max_length:
        encoded += [vocab['<PAD>']] * (max_length - len(encoded))  # Padding
    return encoded[:max_length]  # Truncate to max_length

# Encode all reviews
X_train_encoded = torch.tensor([encode_text(text, vocab) for text in X_train])
X_test_encoded = torch.tensor([encode_text(text, vocab) for text in X_test])
y_train_tensor = torch.nn.functional.one_hot(torch.tensor(y_train.values))
y_test_tensor =  torch.nn.functional.one_hot(torch.tensor(y_test.values))

# Create DataLoader for batching
train_data = TensorDataset(X_train_encoded, y_train_tensor)
test_data = TensorDataset(X_test_encoded, y_test_tensor)

train_loader = DataLoader(train_data, batch_size=500, shuffle=True)
test_loader = DataLoader(test_data, batch_size=500, shuffle=False)

[nltk_data] Downloading package punkt to /home/oru2/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/oru2/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /home/oru2/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


Training samples: 40000, Test samples: 10000


In [27]:
use_cuda = True
device = torch.device("cuda" if use_cuda else "cpu")
batch_size = 500

MAX_GRAD_NORM = 1.2
np.random.seed(42)
torch.manual_seed(42)

In [28]:
from model import SentimentModel
vocab_size = len(vocab)
embed_size = 100
hidden_size = 128
output_size = 2  # Binary classification (positive/negative)
num_layers = 2
del fc_model
fc_model = SentimentModel(vocab_size, embed_size, hidden_size, output_size, num_layers).to(device)
num_epochs = 20



In [29]:
from opacus import PrivacyEngine
model.remove_hooks()
privacy_engine = PrivacyEngine(secure_mode=False)
optimizer = torch.optim.SGD(fc_model.parameters(), lr=1e-2)
model, optimizer, train_loader = privacy_engine.make_private_with_epsilon(
    module=fc_model,
    optimizer=optimizer,
    data_loader=train_loader,
    max_grad_norm=1.5,
    target_delta=1e-5,
    target_epsilon=5,
    epochs=20,
)

/home/oru2/project/env/lib/python3.10/site-packages/opacus/privacy_engine.py:95: UserWarning: Secure RNG turned off. This is perfectly fine for experimentation as it allows for much faster training performance, but remember to turn it on and retrain one last time before production with ``secure_mode`` turned on.
  warnings.warn(
/home/oru2/project/env/lib/python3.10/site-packages/opacus/accountants/analysis/rdp.py:332: UserWarning: Optimal order is the largest alpha. Please consider expanding the range of alphas to get a tighter privacy bound.
  warnings.warn(


In [30]:
def train_model(model, optimizer, train_loader, num_epochs):
    # TODO: implement this function that trains a given model on the MNIST dataset.
    # this is a general-purpose function for both standard training and adversarial training.
    # (toggle enable_defense parameter to switch between training schemes)
    noise_multiplier = 1.1
    lr = 1e-2
    losses = []
    model.train()
    # optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    for epoch in tqdm(range(num_epochs)):
        running_loss = 0.0
        for index, (inputs, labels) in enumerate(train_loader):
            inputs, labels = inputs.to(device), labels.to(device)

            # Zero gradients
            optimizer.zero_grad()

            # Forward pass (Standard Training)
            outputs = model(inputs)
            loss = nn.BCELoss(reduction='none')(outputs.squeeze(), labels.float())
            loss = loss.mean()
            # Adversarial Training (if enabled)
            with torch.no_grad():
                embeddings = model.embedding(inputs)  # Embedding layer forward pass
            adv_inputs = attacks.perturb_data(model, embeddings, labels)  # Generate adversarial inputs

            predictions = model.lstm(adv_inputs)[0][:, -1, :]  # LSTM output
            predictions = torch.sigmoid(model.fc(predictions))  # Final prediction layer
            adv_loss = nn.BCELoss(reduction='none')(predictions.squeeze(), labels.float())
            adv_loss = adv_loss.mean()


            total_loss = loss + 0.5 * adv_loss
            total_loss.backward()
            optimizer.step()

            # Update running loss
            running_loss += total_loss.item()

        
        printstr = (
        f"\t Epoch {epoch}. Loss: {np.mean(running_loss):.6f}"
        )
        if privacy_engine:
            epsilon = privacy_engine.get_epsilon(1e-5)
            printstr += f" | (ε = {epsilon:.2f}, δ = {1e-5})"

        print(printstr)



In [31]:
train_model(model, optimizer, train_loader, num_epochs)
# train_model_dp(fc_model, train_loader, num_epochs)

  0%|          | 0/20 [00:00<?, ?it/s]/home/oru2/project/env/lib/python3.10/site-packages/torch/nn/modules/module.py:1827: FutureWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  self._maybe_warn_non_full_backward_hook(args, result, grad_fn)
  5%|▌         | 1/20 [01:57<37:18, 117.81s/it]

	 Epoch 0. Loss: 83.940964 | (ε = 1.76, δ = 1e-05)


 10%|█         | 2/20 [03:52<34:48, 116.02s/it]

	 Epoch 1. Loss: 83.808544 | (ε = 2.10, δ = 1e-05)


 15%|█▌        | 3/20 [05:52<33:24, 117.93s/it]

...

	 Epoch 17. Loss: 83.335898 | (ε = 4.75, δ = 1e-05)


 95%|█████████▌| 19/20 [36:39<01:55, 115.12s/it]

	 Epoch 18. Loss: 83.327679 | (ε = 4.88, δ = 1e-05)


100%|██████████| 20/20 [38:34<00:00, 115.72s/it]

	 Epoch 19. Loss: 83.318122 | (ε = 5.00, δ = 1e-05)


In [32]:
torch.save(model, 'models/dp-adv.pt')

In [9]:
fc_model = torch.load(f="models/dp.pt")

/tmp/ipykernel_1187474/4042864840.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  fc_model = torch.load(f="models/dp.pt")


In [57]:
correct = 0
model.eval()
for j, (inputs, labels) in enumerate(test_loader):
  inputs = inputs.to(device)
  labels = labels.to(device)
  
  logits = model(inputs)

  prediction = torch.argmax(logits, 1)
  correct += (prediction == torch.argmax(labels, dim=1)).sum().item()
  # print('Batch [{}/{}]'.format(j+1, len(test_loader)))
model.train()
print('Accuracy = {}%'.format(float(correct) * 100 / 10000))

Accuracy = 50.68%


In [75]:
correct = 0
eps = 0.1
# attack = attacks.LSTMPGD(fc_model, epsilon=eps, num_steps=10, embedding_layer=fc_model.embedding)
fc_model.eval()
for j, (inputs, labels) in enumerate(test_loader):
  inputs = inputs.to(device)
  labels = labels.to(device)
  with torch.no_grad():
    embeddings = fc_model.embedding(inputs)
  adv_inputs = attacks.perturb_data(fc_model, embeddings, labels, optimizer)
  # print(adv_images)
  logits = fc_model(inputs)

  # adv_logits = model.lstm(adv_inputs)[0][:, -1, :]
  # adv_logits = torch.sigmoid(fc_model.fc(adv_logits))
  # adv_logits = fc_model(adv_images)

  prediction = torch.argmax(logits, 1)

  # adv_prediction = torch.argmax(adv_logits, 1)


  correct += (prediction == torch.argmax(labels, dim = 1)).sum().item()
  # correct += (adv_prediction == torch.argmax(labels)).sum().item()
  # print('Batch [{}/{}]'.format(j+1, len(test_loader)))
print('Accuracy = {}%'.format(float(correct) * 100 / 20000))

IndexError: pop from empty list

In [76]:
from art.attacks.inference.membership_inference import MembershipInferenceBlackBox
from art.estimators.classification import PyTorchClassifier

In [77]:
from model import SentimentModelNoEmbed
no_embed_model = SentimentModelNoEmbed(embed_size, hidden_size, output_size, num_layers).to(device)

optimizer_ = torch.optim.Adam(no_embed_model.parameters(), lr=1e-2)
model, optimizer, train_loader = privacy_engine.make_private_with_epsilon(
    module=no_embed_model,
    optimizer=optimizer_,
    data_loader=train_loader,
    max_grad_norm=1.5,
    target_delta=1e-5,
    target_epsilon=5,
    epochs=20,
)

/home/oru2/project/env/lib/python3.10/site-packages/opacus/privacy_engine.py:469: UserWarning: You're calling make_private_with_epsilon with non-zero privacy budget already spent. Returned noise_multiplier assumes zero starting point, so your overall privacy budget will be higher.
  warnings.warn(
/home/oru2/project/env/lib/python3.10/site-packages/opacus/accountants/analysis/rdp.py:332: UserWarning: Optimal order is the largest alpha. Please consider expanding the range of alphas to get a tighter privacy bound.
  warnings.warn(


In [78]:
print(model.state_dict().keys())
state_dict = {k: fc_model.state_dict()[k] for k in filter(lambda x: not 'embedding' in x, fc_model.state_dict())}
no_embed_model.load_state_dict(state_dict, strict=False) 


odict_keys(['_module.lstm.weight_ih_l0', '_module.lstm.bias_ih_l0', '_module.lstm.weight_hh_l0', '_module.lstm.bias_hh_l0', '_module.lstm.weight_ih_l1', '_module.lstm.bias_ih_l1', '_module.lstm.weight_hh_l1', '_module.lstm.bias_hh_l1', '_module.lstm.l0.ih.weight', '_module.lstm.l0.ih.bias', '_module.lstm.l0.hh.weight', '_module.lstm.l0.hh.bias', '_module.lstm.l1.ih.weight', '_module.lstm.l1.ih.bias', '_module.lstm.l1.hh.weight', '_module.lstm.l1.hh.bias', '_module.fc.weight', '_module.fc.bias'])


<All keys matched successfully>

In [79]:

criterion = nn.BCELoss()


# Wrap the PyTorch model in ART's PyTorchClassifier
art_classifier = PyTorchClassifier(
    model=no_embed_model,
    loss=criterion,
    optimizer=optimizer_,
    input_shape=(200,),
    nb_classes=2
)
attack_train_size = 10000
attack_test_size = 5000

x_train = train_data.tensors[0]
y_train = train_data.tensors[1].detach().numpy()


x_test = test_data.tensors[0]
y_test = test_data.tensors[1].detach().numpy()

x_train = nn.Embedding(vocab_size, embed_size)(x_train).detach().numpy()
x_test = nn.Embedding(vocab_size, embed_size)(x_test).detach().numpy()

attack = MembershipInferenceBlackBox(estimator=art_classifier, attack_model_type="nn")
attack.fit(x_train[:attack_train_size], y_train[:attack_train_size], x_test[:attack_test_size], y_test[:attack_test_size])


In [81]:
mlp_inferred_train_bb = attack.infer(x_train[attack_train_size:], y_train[attack_train_size:])
mlp_inferred_test_bb = attack.infer(x_test[attack_test_size:], y_test[attack_test_size:])

# check accuracy
mlp_train_acc_bb = np.sum(mlp_inferred_train_bb) / len(mlp_inferred_train_bb)
mlp_test_acc_bb = 1 - (np.sum(mlp_inferred_test_bb) / len(mlp_inferred_test_bb))
mlp_acc_bb = (mlp_train_acc_bb * len(mlp_inferred_train_bb) + mlp_test_acc_bb * len(mlp_inferred_test_bb)) / (len(mlp_inferred_train_bb) + len(mlp_inferred_test_bb))

print(f"Members Accuracy: {mlp_train_acc_bb:.4f}")
print(f"Non Members Accuracy {mlp_test_acc_bb:.4f}")
print(f"Attack Accuracy {mlp_acc_bb:.4f}")

Members Accuracy: 0.9982
Non Members Accuracy 0.8660
Attack Accuracy 0.9793


In [ ]:
privacy_engine.get_epsilon

In [63]:
x_train.detach()

tensor([[[ 1.2008e+00,  1.5396e+00,  1.4179e+00,  ...,  5.5041e-01,
          -4.3193e-01, -2.4356e-01],
         [ 2.2167e-01, -2.7137e-01,  2.1014e-01,  ..., -1.0977e+00,
          -7.6189e-01, -1.6361e+00],
         [-8.4064e-03, -9.4465e-02, -4.9066e-01,  ...,  7.0453e-01,
           6.9287e-01, -1.4205e+00],
         ...,
         [ 7.2725e-02,  7.8139e-01,  6.2422e-01,  ..., -2.6550e+00,
           3.7356e-01, -4.9829e-01],
         [ 7.2725e-02,  7.8139e-01,  6.2422e-01,  ..., -2.6550e+00,
           3.7356e-01, -4.9829e-01],
         [ 7.2725e-02,  7.8139e-01,  6.2422e-01,  ..., -2.6550e+00,
           3.7356e-01, -4.9829e-01]],

        [[ 1.4441e+00,  1.1737e+00, -1.4373e+00,  ..., -1.0749e+00,
          -1.6892e+00, -1.3481e+00],
         [ 6.1772e-01, -6.9171e-01,  1.2618e+00,  ...,  1.9872e+00,
          -3.9961e-01, -5.8853e-01],
         [-1.2826e-01, -4.8372e-01, -5.2664e-01,  ..., -5.0066e-01,
          -1.2826e-01, -1.1033e+00],
         ...,
         [ 7.2725e-02,  7